In [1]:
from pathlib import Path

import pandas as pd
from lightning import pytorch as pl
import ray
from ray import tune
from ray.train import CheckpointConfig, RunConfig, ScalingConfig
from ray.train.lightning import (RayDDPStrategy, RayLightningEnvironment,
                                 RayTrainReportCallback, prepare_trainer)
from ray.train.torch import TorchTrainer
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import FIFOScheduler

from chemprop import data, featurizers, models, nn

/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-11 22:02:59,287	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-11 22:03:02,932	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-11 22:03:06,687	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
chemprop_dir = Path.cwd().parent
train_path = "/home/aih/serra.korkmaz/projects/saturn/SurogateModel/training_data_4sm_chembl/train_data_5ht1b.csv"
test_path = "/home/aih/serra.korkmaz/projects/saturn/SurogateModel/training_data_4sm_chembl/test_data_5ht1b.csv"
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['value'] # list of names of the columns containing targets

hpopt_save_dir = Path.cwd() / "hpopt_5ht1b_chembl" # directory to save hyperopt results
hpopt_save_dir.mkdir(exist_ok=True)

In [4]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [5]:
# Extract SMILES and target values
train_smis = df_train[smiles_column].values
train_ys = df_train[target_columns].values

test_smis = df_test[smiles_column].values
test_ys = df_test[target_columns].values

# Convert data to MoleculeDatapoint format
train_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(train_smis, train_ys)]
test_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(test_smis, test_ys)]


In [6]:
# Initialize featurizer
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

# Create training dataset
train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

# Split validation set from training data (80% train, 20% validation)
split_idx = int(len(train_data) * 0.8)
val_data = train_data[split_idx:]
train_data = train_data[:split_idx]

# Create validation and test datasets
val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data, featurizer)

In [7]:
def train_model(config, train_dset, val_dset, num_workers, scaler):

    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth)
    agg = nn.MeanAggregation()
    output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    ffn = nn.RegressionFFN(output_transform=output_transform, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers)
    batch_norm = True
    metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=20, # number of epochs to train for
        # below are needed for Ray and Lightning integration
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
    )

    trainer = prepare_trainer(trainer)
    trainer.fit(model, train_loader, val_loader)

In [8]:
search_space = {
    "depth": tune.qrandint(lower=2, upper=6, q=1),
    "ffn_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
    "ffn_num_layers": tune.qrandint(lower=1, upper=3, q=1),
    "message_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
}

In [9]:
ray.init()

scheduler = FIFOScheduler()

# Scaling config controls the resources used by Ray
scaling_config = ScalingConfig(
    num_workers=1,
    use_gpu=False, # change to True if you want to use GPU
)

# Checkpoint config controls the checkpointing behavior of Ray
checkpoint_config = CheckpointConfig(
    num_to_keep=1, # number of checkpoints to keep
    checkpoint_score_attribute="val_loss", # Save the checkpoint based on this metric
    checkpoint_score_order="min", # Save the checkpoint with the lowest metric value
)

run_config = RunConfig(
    checkpoint_config=checkpoint_config,
    storage_path=hpopt_save_dir / "ray_results", # directory to save the results
)

ray_trainer = TorchTrainer(
    lambda config: train_model(
        config, train_dset, val_dset, num_workers, scaler
    ),
    scaling_config=scaling_config,
    run_config=run_config,
)

search_alg = HyperOptSearch(
    n_initial_points=1, # number of random evaluations before tree parzen estimators
    random_state_seed=42,
)

# OptunaSearch is another search algorithm that can be used
# search_alg = OptunaSearch()

tune_config = tune.TuneConfig(
    metric="val_loss",
    mode="min",
    num_samples=2, # number of trials to run
    scheduler=scheduler,
    search_alg=search_alg,
    trial_dirname_creator=lambda trial: str(trial.trial_id), # shorten filepaths

)

tuner = tune.Tuner(
    ray_trainer,
    param_space={
        "train_loop_config": search_space,
    },
    tune_config=tune_config,
)

# Start the hyperparameter search
results = tuner.fit()

(RayTrainWorker pid=27866) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=27590) Started distributed worker processes: 
(TorchTrainer pid=27590) - (node_id=353342b5f20a225baf5438ce754f3005a678d693116335532ce6e99c, ip=10.233.0.37, pid=27866) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=27866) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/train/lightning/_lightning_utils.py:262: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
(RayTrainWorker pid=27866) `get_trial_name` is deprecated because the concept of a `Trial` will soon be removed in Ray Train.Ray Train will no longer assume that it's running within a Ray Tune `Trial` in the future. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment vari

Epoch 0:   7%|▋         | 2/29 [00:02<00:33,  0.81it/s, v_num=3.41e+7, train_loss_step=0.996]


(TorchTrainer pid=27867) Started distributed worker processes: 
(TorchTrainer pid=27867) - (node_id=353342b5f20a225baf5438ce754f3005a678d693116335532ce6e99c, ip=10.233.0.37, pid=28099) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=28099) Setting up process group for: env:// [rank=0, world_size=1]


Epoch 0:  48%|████▊     | 14/29 [00:06<00:07,  2.03it/s, v_num=3.41e+7, train_loss_step=0.696]


(RayTrainWorker pid=28099) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/train/lightning/_lightning_utils.py:262: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
(RayTrainWorker pid=28099) `get_trial_name` is deprecated because the concept of a `Trial` will soon be removed in Ray Train.Ray Train will no longer assume that it's running within a Ray Tune `Trial` in the future. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the environment variable: RAY_TRAIN_ENABLE_V2_MIGRATION_WARNINGS=0
(RayTrainWorker pid=28099) GPU available: False, used: False
(RayTrainWorker pid=28099) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=28099) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=28099) Loading `train_dataloader` to estimate number 

...

2025-03-11 22:09:27,295	WARNING experiment_state.py:206 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds and may become a bottleneck. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this warning by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0). Set it to 0 to completely suppress this warning.


(RayTrainWorker pid=27866) 
Epoch 19: 100%|██████████| 29/29 [00:10<00:00,  2.83it/s, v_num=3.41e+7, train_loss_step=0.475, val_loss=0.333, train_loss_epoch=0.378]


(RayTrainWorker pid=27866) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_chembl/ray_results/TorchTrainer_2025-03-11_22-04-38/1951b44b/checkpoint_000019)
(RayTrainWorker pid=27866) `Trainer.fit` stopped: `max_epochs=20` reached.


Epoch 19: 100%|██████████| 29/29 [00:10<00:00,  2.80it/s, v_num=3.41e+7, train_loss_step=0.475, val_loss=0.333, train_loss_epoch=0.378]


2025-03-11 22:09:28,595	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_chembl/ray_results/TorchTrainer_2025-03-11_22-04-38' in 0.0070s.
2025-03-11 22:09:28,600	INFO tune.py:1041 -- Total run time: 289.92 seconds (289.36 seconds for the tuning loop).


In [10]:
results

ResultGrid<[
  Result(
    metrics={'train_loss': 0.37753477692604065, 'train_loss_step': 0.47536128759384155, 'val/rmse': 0.5768219828605652, 'val/mae': 0.4369758665561676, 'val_loss': 0.33272358775138855, 'train_loss_epoch': 0.37753477692604065, 'epoch': 19, 'step': 580},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_chembl/ray_results/TorchTrainer_2025-03-11_22-04-38/1951b44b',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_chembl/ray_results/TorchTrainer_2025-03-11_22-04-38/1951b44b/checkpoint_000019)
  ),
  Result(
    metrics={'train_loss': 0.35938483476638794, 'train_loss_step': 0.4209131598472595, 'val/rmse': 0.5664517879486084, 'val/mae': 0.4320515990257263, 'val_loss': 0.3208676278591156, 'train_loss_epoch': 0.35938483476638794, 'epoch': 19, 'step': 580},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_chembl/ray_results/TorchTr

In [11]:
result_df = results.get_dataframe()
result_df

,train_loss,train_loss_step,val/rmse,val/mae,val_loss,train_loss_epoch,epoch,step,timestamp,checkpoint_dir_name,...,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/train_loop_config/depth,config/train_loop_config/ffn_hidden_dim,config/train_loop_config/ffn_num_layers,config/train_loop_config/message_hidden_dim,logdir
0,0.377535,0.475361,0.576822,0.436976,0.332724,0.377535,19,580,1741727367,checkpoint_000019,...,27590,cpusrv27.scidom.de,10.233.0.37,276.938678,20,2,2000,2,500,1951b44b
1,0.359385,0.420913,0.566452,0.432052,0.320868,0.359385,19,580,1741727343,checkpoint_000019,...,27867,cpusrv27.scidom.de,10.233.0.37,243.204386,20,2,2200,2,400,2a0ae81e


In [12]:
# best configuration
best_result = results.get_best_result()
best_config = best_result.config
best_config['train_loop_config']

{'depth': 2,
 'ffn_hidden_dim': 2200,
 'ffn_num_layers': 2,
 'message_hidden_dim': 400}

In [13]:
# best model checkpoint path
best_result = results.get_best_result()
best_checkpoint_path = Path(best_result.checkpoint.path) / "checkpoint.ckpt"
print(f"Best model checkpoint path: {best_checkpoint_path}")

Best model checkpoint path: /ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_5ht1b_chembl/ray_results/TorchTrainer_2025-03-11_22-04-38/2a0ae81e/checkpoint_000019/checkpoint.ckpt


In [14]:
mpnn = models.MPNN.load_from_checkpoint(best_checkpoint_path)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=400, bias=False)
    (W_h): Linear(in_features=400, out_features=400, bias=False)
    (W_o): Linear(in_features=472, out_features=400, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=400, out_features=2200, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2200, out_features=2200, bias=True)
      )
      (2): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2200, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]

In [15]:
import torch
test_loader = data.build_dataloader(test_dset, shuffle=False)
with torch.inference_mode():
    trainer = pl.Trainer(
        logger=None,
        enable_progress_bar=True,
        accelerator="cpu",
        devices=1
    )
    test_preds = trainer.predict(mpnn, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
SLURM auto-requeueing enabled. Setting signal handlers.
/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 4/4 [00:00<00:00,  7.02it/s]


In [16]:
import numpy as np
test_preds = np.concatenate(test_preds, axis=0)
df_test['preds'] = test_preds
df_test

,smiles,value,preds
0,O=C(Nc1ccc(F)cc1)C(c1ccccc1)N1CCN(c2ccncc2)CC1,9.6,7.054516
1,CN1CCN(c2ccc(N=c3c4c([nH]c5cc(Cl)ccc35)CCCC4)c...,10.2,8.862932
2,Cc1ccccc1C(CC(=O)O)NC(=O)c1cccc(-c2ccc(F)cc2)n1,6.8,9.257415
3,N=c1ncc(-c2nc(N3CCOCC3)nc3c2CCN3C2CCN(C=O)C2)c...,10.2,8.568460
4,CC1Cc2c([nH]c3ccc(F)cc23)C2(CCC(c3ccccc3)(N(C)...,9.8,8.220876
...,...,...,...
195,COC(=O)NC(C(=O)NC(Cc1ccccc1)C(O)CN(OC1CCCC1)S(...,5.5,5.068117
196,Cc1cc2c(cc1Cl)SC(C(=O)c1ccc(Br)cc1)=NS2(=O)=O,6.3,8.308681
197,COc1ccc(C(=O)OC(C=C(C)C)CC(C)C2CCC3(C)C4C(OC)C...,5.8,5.394099
198,Cc1nc2ccccc2nc1N1CC2CN(C(=O)c3ccccc3-c3nc[nH]n...,10.8,8.273340
